In [4]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os

In [ ]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [ ]:
doc=read_doc('documents/')
len(doc)

In [ ]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [ ]:
documents=chunk_data(docs=doc)
len(documents)

In [ ]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

In [ ]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

In [ ]:
## Vector Search DB In Pinecone
pinecone.init(
    api_key="923d5299-ab4c-4407-bfe6-7f439d9a9cb9",
    environment="gcp-starter"
)
index_name="langchainvector"

In [ ]:
index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [ ]:
## Cosine Similarity Retreive Results from VectorDB
## k=2 -> top two query
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)